cuantas veces aparecen los puntos,

las palabras con poco contenido semantico

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/My Drive/ITD/Int Artificial Avanzada/Parte 2/PLN/NLTK/"

# Para visualizar originallTraining.txt
with open(os.path.join(base_path, "originallTraining.txt"), "r", encoding="utf-8") as f:
    for i in range(10):  # Imprime las primeras 10 líneas
        line = f.readline()
import os

base_path = "/content/drive/My Drive/ITD/Int Artificial Avanzada/Parte 2/PLN/NLTK/"

# Para visualizar originallTraining.txt
with open(os.path.join(base_path, "originallTraining.txt"), "r", encoding="utf-8") as f:
    for i in range(10):  # Imprime las primeras 10 líneas
        line = f.readline()
        print(line)

with open(os.path.join(base_path, "originalTest.txt"), "r", encoding="utf-8") as f:
    pass

Mounted at /content/drive
264183816548130816	15140428	positive	Gas by my house hit $3.39!!!! I'm going to Chapel Hill on Sat. :)

264249301910310912	18516728	negative	Iranian general says Israel's Iron Dome can't deal with their missiles (keep talking like that and we may end up finding out)

264105751826538497	147088367	positive	with J Davlar 11th. Main rivals are team Poland. Hopefully we an make it a successful end to a tough week of training tomorrow.

264094586689953794	332474633	negative	Talking about ACT's && SAT's, deciding where I want to go to college, applying to colleges and everything about college stresses me out.

254941790757601280	557103111	negative	They may have a SuperBowl in Dallas, but Dallas ain't winning a SuperBowl. Not with that quarterback and owner. @S4NYC @RasmussenPoll

264169034155696130	382403760	neutral	Im bringing the monster load of candy tomorrow, I just hope it doesn't get all squiched

263192091700654080	344222239	objective-OR-neutral	Apple software

Realice lo siguiente sobre el archivo sin procesar de tweets:

1. Proponga y describa detalle un proceso de preprocesamiento sobre “textos crudos" que maximice el vocabulario en un modelo de aprendizaje supervisado. El proceso debe incluir:
+ Codificación/decodificación (si aplica)
+ Manejo de mayúsculas y minúsculas
+ Uso de patrones Regex
+ Manejo de palabras cerradas
+ Manejo de signos de puntuación y caracteres especiales
+ Manejo de etiquetas del discurso (PoS Tags)
+ Uso de un lematizador/stemming.
+ Entre otros
2. Utilizando el pipeline propuesto, limpie el texto y ocupe alguna de las técnicas de aprendizaje supervisado vistas anteriormente (Naive Bayes, por ejemplo) con una representación básica como Bag of words o One-hot-encoding (también vistas de manera elemental antes) para entrenar un modelo de análisis de sentimientos.
3. Documente distintas pruebas para ver si el uso de un preprocesamiento mejora o empeora el rendimiento de un nuevo modelo.
4. Muestre las conclusiones alcanzadas desde el punto de vista del manejo del lenguaje.

In [ ]:
import pandas as pd

# Define column names
column_names = ['id', 'user_id', 'sentiment', 'text']

# Read training data
training_data = pd.read_csv(os.path.join(base_path, "originallTraining.txt"),
                            sep='\t', header=None, names=column_names)

# Read test data (assuming similar format)
test_data = pd.read_csv(os.path.join(base_path, "originalTest.txt"),
                        sep='\t', header=None, names=column_names)

# Now you can access columns by name:
training_data['text'] # Access the text column
training_data['sentiment'] # Access the sentiment column

,sentiment
0,positive
1,negative
2,positive
3,negative
4,negative
...,...
6147,negative
6148,positive
6149,positive
6150,neutral


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack
import pandas as pd
import os
from gensim.models import Word2Vec
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler
from gensim.corpora import Dictionary
from gensim.matutils import corpus2csc

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    text = ' '.join(tokens)
    return text

def extract_hashtags(text):
    hashtags = re.findall(r"#(\w+)", text)
    return hashtags

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Define column names
column_names = ['id', 'user_id', 'sentiment', 'text']

# Read training data
base_path = "/content/drive/My Drive/ITD/Int Artificial Avanzada/Parte 2/PLN/NLTK/"
training_data = pd.read_csv(os.path.join(base_path, "originallTraining.txt"),
                            sep='\t', header=None, names=column_names)

# Read test data (assuming similar format)
test_data = pd.read_csv(os.path.join(base_path, "originalTest.txt"),
                        sep='\t', header=None, names=column_names)

# Truncate datasets to ensure equal length
min_length = min(len(training_data), len(test_data))
training_data = training_data[:min_length]
test_data = test_data[:min_length]

# Preprocesar los datos de entrenamiento y prueba
training_data['processed_text'] = training_data['text'].apply(preprocess_text)
test_data['processed_text'] = test_data['text'].apply(preprocess_text)

# Extraer hashtags
training_data['hashtags'] = training_data['text'].apply(extract_hashtags)
test_data['hashtags'] = test_data['text'].apply(extract_hashtags)

In [ ]:
# Crear diccionario y Bag of Words con Gensim
texts = [text.split() for text in training_data['processed_text']]
dictionary = Dictionary(texts)
training_data_bow = [dictionary.doc2bow(text.split()) for text in training_data['processed_text']]
test_data_bow = [dictionary.doc2bow(text.split()) for text in test_data['processed_text']]

# Convertir a matriz dispersa
training_data_vectorized = corpus2csc(training_data_bow, num_terms=len(dictionary)).transpose()
test_data_vectorized = corpus2csc(test_data_bow, num_terms=len(dictionary)).transpose()
# Crear características de hashtags
mlb = MultiLabelBinarizer()
hashtag_features_train = mlb.fit_transform(training_data['hashtags'])
hashtag_features_test = mlb.transform(test_data['hashtags'])

# Combinar características
training_data_vectorized = hstack([training_data_vectorized, hashtag_features_train])
test_data_vectorized = hstack([test_data_vectorized, hashtag_features_test])

# Definir los hiperparámetros a ajustar
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'fit_prior': [True, False]
}

# Assuming 'sentiment' is the target variable
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    training_data_vectorized, training_data['sentiment'], test_size=0.2, random_state=42
)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['1game', '1st', '299COMM', '3', '49ers', '4N', '4networking', '5yearsold', '7FactsAboutMyBestFriend', '7ThingsAboutMyBestFriend', 'ACC', 'AFC', 'Arsenal', 'AtrAin', 'BBAU', 'Badtime', 'Believe', 'BestPrimetimeLineup', 'BringBackLMFAO', 'Brownlow2012', 'Bulldogs', 'Bullshifter', 'ByeByeWage', 'CISwrug12', 'Celtics', 'Charlie_Chaplin', 'Clippers', 'Come', 'Contraband', 'CurrentEvents', 'DECALove', 'DeathValley', 'E60', 'EXCITED', 'Excited', 'F2S', 'FAN', 'FIFA13', 'FIREPIOLI', 'Facts', 'FamOverAll', 'FightForMatt', 'FirstTake', 'Flight', 'FootballAsAKid', 'FreeBooth', 'GGMU', 'Generous', 'GoBears', 'GoBroncos', 'GoIndiana', 'GoodTimes', 'Gossip', 'GreatSong', 'HOLLA', 'HOU', 'HalloweenSong', 'HamOnt', 'Happy', 'HappyBirthdayTroianBellisario', 'HarryPotter', 'HatedIt', 'HavePlenty', 'Hawks', 'HerdNation', 'Hokies', 'HowSad', 'ILoveMyBestFriend', 'Idiots', 'Inspiration', 'Intersex',

In [ ]:
# Crear un clasificador Naive Bayes
classifier = MultinomialNB()

# Realizar la búsqueda de cuadrícula para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train_split, y_train_split)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros
best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(X_val_split)
accuracy = accuracy_score(y_val_split, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Mejores hiperparámetros: {'alpha': 10.0, 'fit_prior': False}
Precisión del modelo: 41.54%


In [ ]:
# Predecir las etiquetas para los datos de prueba
y_pred = best_classifier.predict(test_data_vectorized) # Replace X_test with test_data_vectorized

# Mostrar algunas predicciones con su información real
for i in range(10):  # Mostrar las primeras 10 predicciones
    print(f"Texto: {test_data['text'][i]}")
    print(f"Sentimiento real: {test_data['sentiment'][i]}")
    print(f"Sentimiento predicho: {y_pred[i]}")
    print("-" * 20)

Texto: Won the match #getin . Plus, tomorrow is a very busy day, with Awareness Day's and debates. Gulp. Debates...
Sentimiento real: neutral
Sentimiento predicho: positive
--------------------
Texto: Lunch from my new Lil spot ...THE COTTON BOWL ....pretty good#1st#time#will be going back# http://instagr.am/p/RX9939CIv8/ 
Sentimiento real: positive
Sentimiento predicho: positive
--------------------
Texto: SNC Halloween Pr. Pumped. Let's work it for Sunday....Packers vs....who knows or caresn. #SNC #cheerpracticeonhalloween
Sentimiento real: positive
Sentimiento predicho: positive
--------------------
Texto: Manchester United will try to return to winning ways when they face Arsenal in the Premier League at Old Trafford on Saturday.
Sentimiento real: neutral
Sentimiento predicho: positive
--------------------
Texto: Going to a bulls game with Aaliyah & hope next Thursday
Sentimiento real: neutral
Sentimiento predicho: positive
--------------------
Texto: Any Toon Fans with a spare tic

# Conclusiones sobre el manejo del lenguaje en el análisis de sentimientos:

**El preprocesamiento del texto es crucial para el rendimiento del modelo:**

+ La eliminación de signos de puntuación, caracteres especiales y la conversión a minúsculas ayuda a estandarizar el texto y reduce el ruido en los datos.
+ La eliminación de palabras vacías (stop words) elimina términos comunes que no aportan mucha información al análisis de sentimientos.
+ La lematización reduce las palabras a su forma raíz, lo que ayuda a agrupar palabras con significado similar y mejora la precisión del modelo.

**La representación Bag of Words es una técnica básica pero efectiva:**

+ Permite convertir el texto en una representación numérica que el modelo puede entender.
+ Sin embargo, puede perder información sobre el orden de las palabras y el contexto en el que aparecen.

**El modelo Naive Bayes es simple pero puede ser efectivo para el análisis de sentimientos:**
+ Es fácil de implementar y entrenar.
+ Sin embargo, puede ser menos preciso que otros modelos más complejos, como las redes neuronales.

**La precisión del modelo puede mejorarse con un preprocesamiento más avanzado:**
+ Se podrían explorar otras técnicas de preprocesamiento como la corrección ortográfica, la detección de entidades nombradas y el análisis de dependencias.
+ También se podrían utilizar representaciones de texto más avanzadas como word embeddings o modelos de lenguaje pre-entrenados.

**En general, el manejo adecuado del lenguaje es fundamental para obtener buenos resultados en el análisis de sentimientos:**

+ El preprocesamiento del texto ayuda a limpiar y estandarizar los datos.
+ La elección de la representación del texto y el modelo de aprendizaje afecta la precisión del modelo.
+ La experimentación y la evaluación son esenciales para encontrar la mejor configuración para una tarea específica.

**En este caso particular, el modelo logró una precisión de alrededor del 41.54%, lo que indica que hay margen de mejora:**
+ Se podrían explorar técnicas de preprocesamiento más avanzadas y diferentes modelos de aprendizaje para aumentar la precisión.
+ También sería útil analizar los errores del modelo para entender mejor sus limitaciones y encontrar formas de mejorar su rendimiento.

In [ ]:
# Guardar training_data en un archivo CSV
training_data.to_csv(os.path.join(base_path, "training_data_processed.csv"), index=False)

# Guardar test_data en un archivo CSV
test_data.to_csv(os.path.join(base_path, "test_data_processed.csv"), index=False)